In [2]:
import numpy as np
import PIL
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import os
import onnxruntime as rt
import time

def load_labels(filename):
    with open(filename, "r") as f:
        return [line.strip() for line in f.readlines()]

def get_class_labels(output):
    output = np.squeeze(np.float32(output))

    outputoffset = 0 if (output.shape[0] == 1001) else 1
    top_k = output.argsort()[-5:][::-1]
    labels = load_labels("labels.txt")
    for j, k in enumerate(top_k):
        curr_class = f"\n  {j}  {output[k]:08.6f}  {labels[k+outputoffset]} \n"
        classes = classes + curr_class if ("classes" in locals()) else curr_class
    classes = classes.replace("\n", ",")
    return classes

os.environ["TIDL_RT_PERFSTATS"] = "1"

print("Available execution providers : ", rt.get_available_providers())

sess_options = rt.SessionOptions()
sess_options.log_severity_level = 3
#sess_options.enable_profiling = True

model_path = '/root/work/edgeai-tidl-tools/models/public/resnet18_opset9.onnx'
input_file='/root/work/edgeai-tidl-tools/examples/jupyter_notebooks/colab/jet.jpeg'
EP_list = ['TIDLExecutionProvider', "CPUExecutionProvider"]
delegate_options = {}
delegate_options["artifacts_folder"] = '/root/work/edgeai-tidl-tools/model-artifacts/cl-ort-resnet18-v1/artifacts'
delegate_options["tidl_tools_path"] = '/root/work/edgeai-tidl-tools/tidl_tools'
session = rt.InferenceSession(model_path, providers=EP_list, provider_options=[delegate_options, {}],sess_options=sess_options)


# get the name of the first input of the model
input_name = session.get_inputs()[0].name
input_details  = session.get_inputs()
output_details = session.get_outputs()

#Pre-Process input
#image    = Image.open(input_file).resize((224, 224))
#img_data = np.array(image).transpose(2, 0, 1).astype('float32')
#mean_vec   = np.array([0.485, 0.456, 0.406])
#stddev_vec = np.array([0.229, 0.224, 0.225])

height = input_details[0].shape[2]
width = input_details[0].shape[3]
channel = input_details[0].shape[1]
batch = input_details[0].shape[0]
shape = [batch, channel, height, width]
# Prepare the input data
input_data = np.zeros(shape)

image = Image.open(input_file).convert("RGB").resize((width, height), PIL.Image.LANCZOS)
temp_input_data = np.expand_dims(image, axis=0)
temp_input_data = np.transpose(temp_input_data, (0, 3, 1, 2))
input_data[0] = temp_input_data


#norm_img_data = np.zeros(img_data.shape).astype('float32')
#for i in range(img_data.shape[0]):
#    norm_img_data[i,:,:] = (img_data[i,:,:]/255 - mean_vec[i]) / stddev_vec[i]
#input_data = norm_img_data.reshape(1, 3, 224, 224)
input_data = np.uint8(input_data)
#Run inference session
start_time = time.time()
raw_result = session.run(None, {input_name: input_data})
print("\nInference time : ", time.time() - start_time)

classes = get_class_labels(raw_result[0][0])
print(classes)

Available execution providers :  ['TIDLExecutionProvider', 'TIDLCompilationProvider', 'CPUExecutionProvider']
libtidl_onnxrt_EP loaded 0x21e34c50 
Final number of subgraphs created are : 1, - Offloaded Nodes - 52, Total Nodes - 52 

Inference time :  0.004157066345214844
,  0  18.220415  warplane, military plane ,,  1  17.148626  aircraft carrier, carrier, flattop, attack aircraft carrier ,,  2  14.469153  projectile, missile ,,  3  13.665312  space shuttle ,,  4  13.665312  wing ,
